## 2. .NET Resource Management

### Item 11: Understand .NET Resource Management
Memory management (for the managed heap) is completely the responsibility of the garbage collector. Other system resources must be managed by developers.

Two mechanisms help developers control the lifetimes of unmanaged resources: finalizers and the IDisposable interface. A finalizer is a defensive mechanism that ensures that your objects always have a way to release unmanaged resources. Finalizers have many drawbacks, so you also have the IDisposable interface that provides a less intrusive way to return resources to the system in a timely manner.

The garbage collector is responsible for calling a finalizer on an object instance. Therefore, developers cannot determine at compile time exactly when the finalizer will execute. 

In [12]:
class TestClass {
    public TestClass() {
        Console.WriteLine("Constructor runs...");
    }
    
    ~TestClass() {
        Console.WriteLine("Finalizer runs...");
    }
}


class Ignition {
    public static void Engage() {
        var t = new TestClass();
        t = null;
    }
}


Ignition.Engage();

Constructor runs...


In [13]:
System.GC.Collect(); // force the GC to run

Finalizer runs...


[When to implement a finalizer](https://csharp.2000things.com/2012/12/17/737-when-to-implement-a-finalizer/)

Relying on finalizers also introduces performance penalties. Objects that require finalization put a performance drag on the garbage collector.

The GC optimizes its work by limiting how often it examines first- and second-generation objects. Every GC cycle examines generation 0 objects. Roughly one GC out of ten examines the generation 0 and 1 objects. Roughly one GC cycle out of 100 examines all objects. Think about finalization and its cost again: An object that requires finalization might stay in memory for nine GC cycles more than it would if it did not require finalization.

Finalizers are not a good solution. Yet you still need to free resources. You address these issues using the IDisposable interface and the standard dispose pattern.

### Item 12: Prefer Member Initializers to Assignment Statements
Constructing a member variable when you declare that variable is natural in C#. Just initialize the variable when you declare it.

In [15]:
using System;
using System.Collections.Generic;

class MyClass {
    private List<string> labels = new List<string>();
}

var c = new MyClass();

Regardless of the number of constructors you eventually add to the MyClass type, labels will be initialized properly. Using initializers is the simplest way to avoid uninitialized variables in your types, but it’s not perfect. In three cases, you should not use the initializer syntax.

The first is when you are initializing the object to 0, or null.

The second inefficiency comes when you create multiple initializations for the same object. You should use the initializer syntax only for variables that receive the same initialization in all constructors.

The final reason to move initialization into the body of a constructor is to facilitate exception handling. 

### Item 13: Use Proper Initialization for Static Class Members
A static constructor is a special function that executes before any other methods, variables, or properties defined in that class are accessed for the first time. You use this function to initialize static variables, enforce the singleton pattern, or perform any other necessary work before a class is usable.

In [20]:
public class MySingleton {
    private static readonly MySingleton theOneAndOnly = new MySingleton();
    
    public static MySingleton TheOnly {
        get{return theOneAndOnly;}
    } 
}

var ms = MySingleton.TheOnly;
Console.WriteLine(ms.GetType().Name);

MySingleton


The CLR calls your [static constructor](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/classes-and-structs/static-constructors) automatically before your type is first accessed in an application space (an AppDomain). You can define only one static constructor, and it must not take any arguments.

### Item 14: Minimize Duplicate Initialization Logic
Constructor initializers allow one constructor to call another constructor. This example shows a simple usage.

In [28]:
class ImportantData {}

class MyClass {
    private List<ImportantData> coll;
    private string name;
    
    public MyClass(): this(0, "") {}
    
    public MyClass(int initialCount): this(initialCount, string.Empty) {}
    
    public MyClass(int initialCount, string name) {
        coll = (initialCount > 0) ?
            new List<ImportantData>(initialCount):
            new List<ImportantData>();
        this.name = name;
    }
    
    public override string ToString() {
        return "Name '" + this.name + "' with coll length " + this.coll.Count;
    }
}


In [29]:
var mc = new MyClass();
Console.WriteLine(mc.ToString());

Name '' with coll length 0


In [32]:
var mc = new MyClass(10, "Simon");
Console.WriteLine(mc.ToString());

Name 'Simon' with coll length 0


C# 4.0 added default parameters, which you can use to minimize the duplicated code in constructors. You could replace all the different constructors for MyClass with one constructor that specifies default values for all or many of the values.

In [35]:
class MyClass {
    private List<ImportantData> coll;
    private string name;
    
    public MyClass() : this(0, string.Empty) {}
    
    public MyClass(int initialCount = 0, string name = "") {
        coll = (initialCount > 0) ?
            new List<ImportantData>(initialCount):
            new List<ImportantData>();
        this.name = name;
    }
    
    public override string ToString() {
        return "Name '" + this.name + "' with coll length " + this.coll.Count;
    }
}

In [36]:
var mc = new MyClass(10, "Simon");
Console.WriteLine(mc.ToString());

Name 'Simon' with coll length 0


### Item 15: Avoid Creating Unnecessary Objects
One very common bad practice is to allocate GDI objects in a Windows paint handler. OnPaint() gets called frequently. Every time it gets called, you create another Font object that contains the exact same settings. The garbage collector needs to clean those up for you. Among the conditions that the GC uses to determine when to run are the amount of memory allocated and the frequency of memory allocations. More allocations mean more pressure on the GC, causing it to run more often. That's incredibly inefficient.

Instead, promote the Font object from a local variable to a member variable. Reuse the same font each time you paint the window.

### Item 16: Never Call Virtual Functions in Constructors
Virtual functions exhibit strange behaviors during the construction of an object. An object is not completely created until all constructors have executed. In the meantime, virtual functions may not behave the way you'd like or expect.

### Item 17: Implement the Standard Dispose Pattern
A standard pattern is used throughout the .NET Framework for disposing of unmanaged resources. The users of your type will expect you to follow this standard pattern. The standard dispose idiom frees your unmanaged resources using the IDisposable interface when clients remember, and it uses the finalizer defensively when clients forget. It works with the garbage collector to ensure that your objects pay the performance penalty associated with finalizers only when necessary.

The root base class in the class hierarchy should do the following:
-  It should implement the IDisposable interface to free resources.

-  It should add a finalizer as a defensive mechanism if and only if your class directly contains an unmanaged resource.

-  Both Dispose and the finalizer (if present) delegate the work of freeing resources to a virtual method that derived classes can override for their own resource management needs.


The derived classes need to:
-  Override the virtual method only when the derived class must free its own resources.
-  Implement a finalizer if and only if one of its direct member fields is an unmanaged resource.  -  Remember to call the base class version of the function

In [40]:
public interface IDisposable {
    void Dispose();
}

public class MyResourceHog: IDisposable {
    // flag for already disposed
    private bool alreadyDisposed = false;
    
    
    // implementation of IDisposable
    // Call the virtual Dispose method
    // Suppress Finalization
    public void Dispose() {
        Console.WriteLine("Calling MyResourceHog: public void Dispose()");
        Dispose(true);
        System.GC.SuppressFinalize(this);
    }
    
    // virtual dispose method
    protected virtual void Dispose(bool isDisposing) {
        Console.WriteLine("Calling MyResourceHog: public virtual void Dispose()");
        if (alreadyDisposed) {
            return;
        }
        if (isDisposing) {
            // free managed resources here
        }
        alreadyDisposed = true;
    }
    
    public void ExampleMethod() {
        Console.WriteLine("Calling MyResourceHog: public void ExampleMethod()");
        if (alreadyDisposed) {
            throw new ObjectDisposedException(
                "MyResourceHog",
                "called ExampleMethod on Disposed object"
            );
        }
        // ...
    }
}


// If a derived class needs to perform additional cleanup,
// it implements the protected Dispose method.
class DerivedResourceHog : MyResourceHog {
    private bool disposed = false;
    
    protected override void Dispose(bool isDisposing) {
        Console.WriteLine("Calling DerivedResourceHog: protected override void Dispose()");
        if (disposed) {
            return;
        }
        if (isDisposing) {
            // free managed resources here
        }
        
        // free unmanaged resources here
        // Base class is responsible for calling GC.SuppressFinalize()
        base.Dispose(isDisposing);
        
        disposed = true;
    }
}


In [46]:
var drh = new DerivedResourceHog();
drh.Dispose();

Calling MyResourceHog: public void Dispose()
Calling DerivedResourceHog: protected override void Dispose()
Calling MyResourceHog: public virtual void Dispose()


***